In [40]:
from src.utils import get_solar
from src.CONSTANTS import *
import pandas as pd

pd.set_option('display.max_columns', None)

In [25]:
bounding_box = pd.read_csv(get_file_path("bounding_boxes"), dtype={"FIPS State": str, "FIPS County": str})
FIPS = pd.read_csv("../../data/extras/US_FIPS_Codes.csv", dtype={"FIPS State": str, 'FIPS County': str})

In [99]:
# load suitabilities
county_suitability = pd.read_csv("../../data/suitability_scores/suitability_scores_county.csv")
bg_suitability = pd.read_csv("../../data/suitability_scores/suitability_scores_bg.csv", dtype={"GEOID":str, "STATEFP": str, 'COUNTYFP':str, 'TRACTCE':str, 'BLKGRPCE':str})
project_suitability = pd.read_csv("../../data/suitability_scores/suitability_scores_project.csv", dtype={"GEOID":str, "STATEFP": str, 'COUNTYFP':str, 'TRACTCE':str, 'BLKGRPCE':str})

# Get county and statenames in project suitablity
project_suitability = project_suitability.merge(FIPS, left_on=['STATEFP', 'COUNTYFP'], right_on=['FIPS State', 'FIPS County'], how='left').drop(columns=['FIPS State', "FIPS County", 'geometry'])

mapper = {
    '110': 'Hartford', '190': 'Fairfield', '170': 'Litchfield', 
    '140': 'Middlefield', '120': 'New Haven', '130': 'Tolland',
    '160': 'Windham', '180': 'New London', '150': 'New London'
}

# rename counties for Connecticut by using the mapper on state connecticut only using the COUNTYFP
project_suitability['County Name'] = project_suitability.apply(
    lambda x: mapper[x['COUNTYFP']] if x['STATEFP'] == '09' else x['County Name'], axis=1
)

# also replace State with Connecticut
project_suitability.loc[project_suitability['STATEFP'] == '09', 'State'] = 'Connecticut'

In [100]:
county_project = project_suitability.merge(county_suitability, left_on=['State', 'County Name'], right_on=['State', 'County Name'], how='inner', suffixes=('_project', '_county'))

In [101]:
# get nan rows
county_project.to_csv("county_project.csv")

In [102]:
bg_project = project_suitability.merge(bg_suitability, left_on=['State', 'County Name', 'TRACTCE', 'BLKGRPCE'], right_on=['State', 'County Name', 'TRACTCE', 'BLKGRPCE'], how='inner', suffixes=('_project', '_bg'))

In [104]:
bg_project.to_csv("bg_project.csv")